# Data Cleaning

---

This notebook was created to take our initial dataset, "Public Company ESG Ratings Dataset", which we obtained from Kaggle.com (https://www.kaggle.com/datasets/alistairking/public-company-esg-ratings-dataset) and use it to make an API call to add additional information for analysis.

We gathered performance data on the companies contained in the set by looping through a series of ticker symbols and pulling from the alphavantage.co API. Specific URL’s were used to pull both annual percentage change values and earnings per share (EPS) for as many of the ticker symbols as possible. 

Once the API calls were completed, the data was converted into dataframes, evaluated and merged together. With our initial dataset. This dataset was converted to a CSV file and exported to a folder for use in the primary jupyter notebook we used for our analysis. 



## sources

#### https://www.alphavantage.co/

#### https://www.kaggle.com/datasets/alistairking/public-company-esg-ratings-dataset




In [2]:
# Dependencies and Setup

import pandas as pd
import requests
import json


# Import our Alpha Advantage API key
from api_keys import alpha_api_key


# Bring in initial ESG dataset from Kaggle.com
esg_data_path = "csv_files/data.csv"


# Converting csv to dataframe
esg_data = pd.read_csv(esg_data_path)


# Displaying the data table for preview
esg_data.head()

,ticker,name,currency,exchange,industry,logo,weburl,environment_grade,environment_level,social_grade,...,governance_grade,governance_level,environment_score,social_score,governance_score,total_score,last_processing_date,total_grade,total_level,cik
0,dis,Walt Disney Co,USD,"NEW YORK STOCK EXCHANGE, INC.",Media,https://static.finnhub.io/logo/ef50b4a2b263c84...,https://thewaltdisneycompany.com/,A,High,BB,...,BB,Medium,510,316,321,1147,19-04-2022,BBB,High,1744489
1,gm,General Motors Co,USD,"NEW YORK STOCK EXCHANGE, INC.",Automobiles,https://static.finnhub.io/logo/9253db78-80c9-1...,https://www.gm.com/,A,High,BB,...,B,Medium,510,303,255,1068,17-04-2022,BBB,High,1467858
2,gww,WW Grainger Inc,USD,"NEW YORK STOCK EXCHANGE, INC.",Trading Companies and Distributors,https://static.finnhub.io/logo/f153dcda-80eb-1...,https://www.grainger.com/,B,Medium,BB,...,B,Medium,255,385,240,880,19-04-2022,BB,Medium,277135
3,mhk,Mohawk Industries Inc,USD,"NEW YORK STOCK EXCHANGE, INC.",Consumer products,https://static.finnhub.io/logo/26868a62-80ec-1...,https://mohawkind.com/,A,High,B,...,BB,Medium,570,298,303,1171,18-04-2022,BBB,High,851968
4,lyv,Live Nation Entertainment Inc,USD,"NEW YORK STOCK EXCHANGE, INC.",Media,https://static.finnhub.io/logo/1cd144d2-80ec-1...,https://www.livenationentertainment.com/,BBB,High,BB,...,B,Medium,492,310,250,1052,18-04-2022,BBB,High,1335258


In [3]:
# Looking at the counts of all the dataframes columns
esg_data.count()


ticker                  722
name                    722
currency                722
exchange                722
industry                709
logo                    702
weburl                  706
environment_grade       722
environment_level       722
social_grade            722
social_level            722
governance_grade        722
governance_level        722
environment_score       722
social_score            722
governance_score        722
total_score             722
last_processing_date    722
total_grade             722
total_level             722
cik                     722
dtype: int64

In [4]:
# Looking at the number of unique values
#A number of entries are missing values for things like website or logo, etc. beacuse that info isn't available
#Things like currency of exchange have lower values because the companies are only traded on 2 different exchanges, for example
esg_data.nunique()

ticker                  722
name                    722
currency                  5
exchange                  2
industry                 47
logo                    702
weburl                  706
environment_grade         5
environment_level         3
social_grade              6
social_level              4
governance_grade          4
governance_level          3
environment_score       197
social_score            186
governance_score         80
total_score             441
last_processing_date     30
total_grade               4
total_level               2
cik                     722
dtype: int64

In [5]:
#This was an example API pull that we used to determine what data we could obtain and identify how to pull the values we were looking for
#ticker "cop" example trial of API pull, using TIME_SERIES_MONTHLY function
api_key = alpha_api_key
url = "https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol=cop&apikey=SU7P624CW7MEDZWK"

stock_info_return = requests.get(url)
data = stock_info_return.json()
data

{'Meta Data': {'1. Information': 'Monthly Prices (open, high, low, close) and Volumes',
  '2. Symbol': 'cop',
  '3. Last Refreshed': '2024-04-05',
  '4. Time Zone': 'US/Eastern'},
 'Monthly Time Series': {'2024-04-05': {'1. open': '128.0000',
   '2. high': '133.8400',
   '3. low': '126.7500',
   '4. close': '133.5200',
   '5. volume': '19139110'},
  '2024-03-28': {'1. open': '113.8000',
   '2. high': '128.6400',
   '3. low': '110.8500',
   '4. close': '127.2800',
   '5. volume': '120434107'},
  '2024-02-29': {'1. open': '112.3400',
   '2. high': '115.2900',
   '3. low': '108.8400',
   '4. close': '112.5400',
   '5. volume': '120426852'},
  '2024-01-31': {'1. open': '117.1700',
   '2. high': '121.2400',
   '3. low': '105.7701',
   '4. close': '111.8700',
   '5. volume': '110673689'},
  '2023-12-29': {'1. open': '115.0300',
   '2. high': '119.6400',
   '3. low': '109.7000',
   '4. close': '116.0700',
   '5. volume': '142651731'},
  '2023-11-30': {'1. open': '119.4800',
   '2. high': '123

## Obtaining 5 Year Percent Change Data

In [6]:
# Set base URL and parameters
url = "https://www.alphavantage.co/query?"
function = "TIME_SERIES_MONTHLY"
api_key = alpha_api_key


# Creating a list to store data from the API
percent_change_data = []

# Creating a series of ticker symbols to loop through 
esg_ticker_series = esg_data['ticker']



print("Beginning Data Retrieval     ")
print("-----------------------------")

# Adding a counter
found_count = 0
not_found = 0

# For loop that takes the ticker symbols from the ticker series, converts to JSON and gathers the data we're looking for
for stock in esg_ticker_series:
    
    # Created a try/except block to allow the code to continue to run if it was unable to find a ticker symbol
    try:
        percent_url = f'{url}function={function}&symbol={stock}&apikey={api_key}'
        percent_info_return = requests.get(percent_url)
        data = percent_info_return.json()
        five_yr_start = data['Monthly Time Series']['2019-03-29']['4. close']
        five_yr_end = data['Monthly Time Series']['2024-03-28']['4. close']
        perc_change = (((float(five_yr_end))-(float(five_yr_start)))/(float(five_yr_start)))*100
        annual_change = ((((float(five_yr_end))-(float(five_yr_start)))/(float(five_yr_start)))*100)/5

        # Appending percent_change_data list with the new data
        percent_change_data.append({"ticker": stock,
                                    "5 YR Open": five_yr_start,
                                    "5 YR Close": five_yr_end,
                                    "Percentage Change": perc_change,
                                    "Annualized % Change": annual_change})

        
        found_count += 1
        print(f'Record {found_count}: Data found for {stock}')

    except:
        print(f'skipping {stock}, data not found')
        not_found += 1

# We utilized our counters to get an idea what percentage of the desired data we captured
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")
print(f'{found_count} Records found successfully')
print("-----------------------------")
print(f'{not_found} Records not found')
print("-----------------------------")
success_rate = (found_count/(found_count+not_found))*100
print(f'{success_rate}% of total records found')
print("-----------------------------")


Beginning Data Retrieval     
-----------------------------
Record 1: Data found for dis
Record 2: Data found for gm
Record 3: Data found for gww
Record 4: Data found for mhk
Record 5: Data found for lyv
Record 6: Data found for lvs
Record 7: Data found for clx
Record 8: Data found for aacg
Record 9: Data found for aal
Record 10: Data found for aame
Record 11: Data found for aaoi
Record 12: Data found for aaon
Record 13: Data found for aapl
skipping aatc, data not found
skipping aaww, data not found
skipping aaci, data not found
Record 14: Data found for aadi
Record 15: Data found for abeo
skipping abnb, data not found
Record 16: Data found for abio
skipping abmd, data not found
skipping abos, data not found
skipping absi, data not found
skipping abtx, data not found
Record 17: Data found for abus
skipping abvc, data not found
Record 18: Data found for acad
skipping acac, data not found
skipping accd, data not found
Record 19: Data found for acet
skipping abcl, data not found
Record 20

In [7]:
#Show the newly created DataFrame
percent_change_df = pd.DataFrame(percent_change_data)
percent_change_df.head()

,ticker,5 YR Open,5 YR Close,Percentage Change,Annualized % Change
0,dis,111.0300,122.3600,10.204449,2.040890
1,gm,37.1000,45.3500,22.237197,4.447439
2,gww,300.9300,1017.3000,238.052039,47.610408
3,mhk,126.1500,130.8900,3.757432,0.751486
4,lyv,63.5400,105.7700,66.462071,13.292414


In [8]:
#This was an example API pull that we used to determine what data we could obtain and identify how to pull the values we were looking for
#ticker "cop" example trial of API pull, using EARNINGS function

url = f"https://www.alphavantage.co/query?function=EARNINGS&symbol=cop&apikey=SU7P624CW7MEDZWK"

stock_info_return = requests.get(url)
data = stock_info_return.json()
data

{'symbol': 'COP',
 'annualEarnings': [{'fiscalDateEnding': '2024-03-31', 'reportedEPS': '0'},
  {'fiscalDateEnding': '2023-12-31', 'reportedEPS': '8.78'},
  {'fiscalDateEnding': '2022-12-31', 'reportedEPS': '13.49'},
  {'fiscalDateEnding': '2021-12-31', 'reportedEPS': '6'},
  {'fiscalDateEnding': '2020-12-31', 'reportedEPS': '-0.97'},
  {'fiscalDateEnding': '2019-12-31', 'reportedEPS': '3.59'},
  {'fiscalDateEnding': '2018-12-31', 'reportedEPS': '4.54'},
  {'fiscalDateEnding': '2017-12-31', 'reportedEPS': '0.73'},
  {'fiscalDateEnding': '2016-12-31', 'reportedEPS': '-2.66'},
  {'fiscalDateEnding': '2015-12-31', 'reportedEPS': '-1.39'},
  {'fiscalDateEnding': '2014-12-31', 'reportedEPS': '5.31'},
  {'fiscalDateEnding': '2013-12-31', 'reportedEPS': '5.7'},
  {'fiscalDateEnding': '2012-12-31', 'reportedEPS': '6.11'},
  {'fiscalDateEnding': '2011-12-31', 'reportedEPS': '8.77'},
  {'fiscalDateEnding': '2010-12-31', 'reportedEPS': '5.96'},
  {'fiscalDateEnding': '2009-12-31', 'reportedEPS': 

## Obtaining Earnings Per Share Data

In [9]:

# Set base URL and parameters
url = "https://www.alphavantage.co/query?"
function = "EARNINGS"
api_key = alpha_api_key

# Creating a list to store data from the API
eps_data = []

# Creating a series of ticker symbols to loop through 
esg_stock_list = esg_data['ticker']


print("Beginning Data Retrieval     ")
print("-----------------------------")

# Adding a counter
found_count = 0
not_found = 0

# For loop that takes the ticker symbols from the ticker series, converts to JSON and gathers the data we're looking for
for stock in esg_stock_list:
    
    eps_url = f'https://www.alphavantage.co/query?function={function}&symbol={stock}&apikey={api_key}'

    # Created a try/except block to allow the code to continue to run if it was unable to find a ticker symbol
    try:
        eps_info_return = requests.get(eps_url)
        data = eps_info_return.json()
        eps_2023 = data['annualEarnings'][1]['reportedEPS']
        eps_2022 = data['annualEarnings'][2]['reportedEPS']
        eps_2021 = data['annualEarnings'][3]['reportedEPS']
        eps_2020 = data['annualEarnings'][4]['reportedEPS']
        eps_2019 = data['annualEarnings'][5]['reportedEPS']
        avg_eps = ((float(eps_2023)) + (float(eps_2022)) + (float(eps_2021)) + (float(eps_2020)) + (float(eps_2019)))/5



        # Appending eps_data list with the new data
        eps_data.append({"ticker": stock,
                            "eps 2019": eps_2019, 
                            "eps 2020": eps_2020, 
                            "eps 2021": eps_2021, 
                            "eps 2022": eps_2022, 
                            "eps 2023": eps_2023,
                            "AVG Annual EPS": avg_eps
                           })
        
        found_count += 1
        print(f'Record {found_count}: Data found for {stock}')
    
    except:
        print(f'skipping {stock}, data not found')
        not_found += 1


# We utilized our counters to get an idea what percentage of the desired data we captured
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")
print(f'{found_count} Records found successfully')
print("-----------------------------")
print(f'{not_found} Records not found')
print("-----------------------------")
success_rate = (found_count/(found_count+not_found))*100
print(f'{success_rate}% of total records found')
print("-----------------------------")
   


Beginning Data Retrieval     
-----------------------------
Record 1: Data found for dis
Record 2: Data found for gm
Record 3: Data found for gww
Record 4: Data found for mhk
Record 5: Data found for lyv
Record 6: Data found for lvs
Record 7: Data found for clx
Record 8: Data found for aacg
Record 9: Data found for aal
Record 10: Data found for aame
Record 11: Data found for aaoi
Record 12: Data found for aaon
Record 13: Data found for aapl
skipping aatc, data not found
skipping aaww, data not found
skipping aaci, data not found
Record 14: Data found for aadi
Record 15: Data found for abeo
skipping abnb, data not found
Record 16: Data found for abio
skipping abmd, data not found
Record 17: Data found for abos
Record 18: Data found for absi
skipping abtx, data not found
Record 19: Data found for abus
Record 20: Data found for abvc
Record 21: Data found for acad
skipping acac, data not found
skipping accd, data not found
Record 22: Data found for acet
Record 23: Data found for abcl
Recor

In [10]:
#Show the newly created DataFrame
eps_df = pd.DataFrame(eps_data)
eps_df


,ticker,eps 2019,eps 2020,eps 2021,eps 2022,eps 2023,AVG Annual EPS
0,dis,5.87,2.01,2.28,3.53,3.77,3.492
1,gm,4.3,4.88,7.09,7.6,7.64,6.302
2,gww,17.29,16.17,19.84,29.67,36.65,23.924
3,mhk,10.02,8.83,14.84,12.85,9.19,11.146
4,lyv,-0.07,-8.1,-3.01,0.79,1.3,-1.818
...,...,...,...,...,...,...,...
620,hgv,2.43,0.44,1.55,2.94,2.8,2.032
621,hi,2.45,2.57,3.79,3.92,3.52,3.250
622,hubs,-1.27,-1.91,-1.67,-2.35,-1.52,-1.744
623,hesm,1.22,1.31,1.76,2.01,2.09,1.678


In [11]:
#We merged these two newly created dataframes together 

esg_merge_1 = pd.merge(percent_change_df, eps_df, how="left", on=["ticker", "ticker"])
esg_merge_1.head()

#Investigate and drop as needed

,ticker,5 YR Open,5 YR Close,Percentage Change,Annualized % Change,eps 2019,eps 2020,eps 2021,eps 2022,eps 2023,AVG Annual EPS
0,dis,111.0300,122.3600,10.204449,2.040890,5.87,2.01,2.28,3.53,3.77,3.492
1,gm,37.1000,45.3500,22.237197,4.447439,4.3,4.88,7.09,7.6,7.64,6.302
2,gww,300.9300,1017.3000,238.052039,47.610408,17.29,16.17,19.84,29.67,36.65,23.924
3,mhk,126.1500,130.8900,3.757432,0.751486,10.02,8.83,14.84,12.85,9.19,11.146
4,lyv,63.5400,105.7700,66.462071,13.292414,-0.07,-8.1,-3.01,0.79,1.3,-1.818


In [12]:
esg_merge_1.count()

ticker                 599
5 YR Open              599
5 YR Close             599
Percentage Change      599
Annualized % Change    599
eps 2019               589
eps 2020               589
eps 2021               589
eps 2022               589
eps 2023               589
AVG Annual EPS         589
dtype: int64

In [13]:
#dropping incomplete records

esg_merge_1.dropna(how="any")
esg_merge_1.count()

#This is not making sense

ticker                 599
5 YR Open              599
5 YR Close             599
Percentage Change      599
Annualized % Change    599
eps 2019               589
eps 2020               589
eps 2021               589
eps 2022               589
eps 2023               589
AVG Annual EPS         589
dtype: int64

In [14]:
#Final Merge

final_merge = pd.merge(esg_merge_1, esg_data, how="left", on=["ticker", "ticker"])
final_merge.head()

,ticker,5 YR Open,5 YR Close,Percentage Change,Annualized % Change,eps 2019,eps 2020,eps 2021,eps 2022,eps 2023,...,governance_grade,governance_level,environment_score,social_score,governance_score,total_score,last_processing_date,total_grade,total_level,cik
0,dis,111.0300,122.3600,10.204449,2.040890,5.87,2.01,2.28,3.53,3.77,...,BB,Medium,510,316,321,1147,19-04-2022,BBB,High,1744489
1,gm,37.1000,45.3500,22.237197,4.447439,4.3,4.88,7.09,7.6,7.64,...,B,Medium,510,303,255,1068,17-04-2022,BBB,High,1467858
2,gww,300.9300,1017.3000,238.052039,47.610408,17.29,16.17,19.84,29.67,36.65,...,B,Medium,255,385,240,880,19-04-2022,BB,Medium,277135
3,mhk,126.1500,130.8900,3.757432,0.751486,10.02,8.83,14.84,12.85,9.19,...,BB,Medium,570,298,303,1171,18-04-2022,BBB,High,851968
4,lyv,63.5400,105.7700,66.462071,13.292414,-0.07,-8.1,-3.01,0.79,1.3,...,B,Medium,492,310,250,1052,18-04-2022,BBB,High,1335258


In [15]:
#Final Drop

esg_complete = final_merge.dropna(subset = ["AVG Annual EPS"])

esg_complete.count()

ticker                  589
5 YR Open               589
5 YR Close              589
Percentage Change       589
Annualized % Change     589
eps 2019                589
eps 2020                589
eps 2021                589
eps 2022                589
eps 2023                589
AVG Annual EPS          589
name                    589
currency                589
exchange                589
industry                589
logo                    584
weburl                  587
environment_grade       589
environment_level       589
social_grade            589
social_level            589
governance_grade        589
governance_level        589
environment_score       589
social_score            589
governance_score        589
total_score             589
last_processing_date    589
total_grade             589
total_level             589
cik                     589
dtype: int64

In [16]:
#Export to CSV Files folder

esg_complete.to_csv("csv_files/esg_complete.csv")